Merry Christmas everyone! 

[Advent of Code day 15](http://adventofcode.com/2018/day/15)

I wasn't going to bother with this one, but it's been nagging at me, so I'll give it another spin. And it's good practice for generators...

## Part 1

Bluuurggh, the fact that this one takes so long to describe is a bit worrying, and that's before I've even started reading it...

Parse the input. 

In [1]:
import numpy as np

In [28]:
test_input_str='''#########
#G..G..G#
#.......#
#.......#
#G..E..G#
#.......#
#.......#
#G..G..G#
#########'''

In [84]:

def parse_input(str_in):
    map_ls=[]
    elves_ls=[]
    goblins_ls=[]
    walls_ls=[]

    for (y, nl) in enumerate(str_in.splitlines()):
        map_ls.append([{'#':'#', 'E':'.', 'G':'.', '.':'.'}[c] for c in nl])
        elves_ls.extend([(x, y) for (x, c) in enumerate(nl) if nl[x]=='E'])
        goblins_ls.extend([(x, y) for (x, c) in enumerate(nl) if nl[x]=='G'])
        walls_ls.extend([(x, y) for (x, c) in enumerate(nl) if nl[x]=='#'])
    
    return {'map':map_ls,
            'elves':elves_ls,
            'goblins':goblins_ls,
            'walls':walls_ls}

parse_input(test_input_str)

{'elves': [(4, 4)],
 'goblins': [(1, 1), (4, 1), (7, 1), (1, 4), (7, 4), (1, 7), (4, 7), (7, 7)],
 'map': [['#', '#', '#', '#', '#', '#', '#', '#', '#'],
  ['#', '.', '.', '.', '.', '.', '.', '.', '#'],
  ['#', '.', '.', '.', '.', '.', '.', '.', '#'],
  ['#', '.', '.', '.', '.', '.', '.', '.', '#'],
  ['#', '.', '.', '.', '.', '.', '.', '.', '#'],
  ['#', '.', '.', '.', '.', '.', '.', '.', '#'],
  ['#', '.', '.', '.', '.', '.', '.', '.', '#'],
  ['#', '.', '.', '.', '.', '.', '.', '.', '#'],
  ['#', '#', '#', '#', '#', '#', '#', '#', '#']],
 'walls': [(0, 0),
  (1, 0),
  (2, 0),
  (3, 0),
  (4, 0),
  (5, 0),
  (6, 0),
  (7, 0),
  (8, 0),
  (0, 1),
  (8, 1),
  (0, 2),
  (8, 2),
  (0, 3),
  (8, 3),
  (0, 4),
  (8, 4),
  (0, 5),
  (8, 5),
  (0, 6),
  (8, 6),
  (0, 7),
  (8, 7),
  (0, 8),
  (1, 8),
  (2, 8),
  (3, 8),
  (4, 8),
  (5, 8),
  (6, 8),
  (7, 8),
  (8, 8)]}

OK, so I need to write a bfs. Let's write a generator which at each stage, returns the coordinates of the current point, and the path taken to get there.

In [52]:
import itertools as it
import pyrsistent as pyr
import collections as coll

In [81]:
m= [['#', '#', '#', '#', '#', '#', '#', '#', '#'],
  ['#', '.', '.', '.', '.', '.', '.', '.', '#'],
  ['#', '.', '.', '.', '.', '.', '.', '.', '#'],
  ['#', '.', '.', '.', '.', '.', '.', '.', '#'],
  ['#', '#', '.', '.', '.', '.', '.', '.', '#'],
  ['#', '.', '.', '.', '.', '.', '.', '.', '#'],
  ['#', '.', '.', '.', '.', '.', '.', '.', '#'],
  ['#', '.', '.', '.', '.', '.', '.', '.', '#'],
  ['#', '#', '#', '#', '#', '#', '#', '#', '#']]

m

[['#', '#', '#', '#', '#', '#', '#', '#', '#'],
 ['#', '.', '.', '.', '.', '.', '.', '.', '#'],
 ['#', '.', '.', '.', '.', '.', '.', '.', '#'],
 ['#', '.', '.', '.', '.', '.', '.', '.', '#'],
 ['#', '#', '.', '.', '.', '.', '.', '.', '#'],
 ['#', '.', '.', '.', '.', '.', '.', '.', '#'],
 ['#', '.', '.', '.', '.', '.', '.', '.', '#'],
 ['#', '.', '.', '.', '.', '.', '.', '.', '#'],
 ['#', '#', '#', '#', '#', '#', '#', '#', '#']]

In [82]:
for 

SyntaxError: invalid syntax (<ipython-input-82-4c1afb746712>, line 1)

In [105]:
def walk_search_space(start_state, daughters_fn, success_fn, fail_fn, strategy='bfs'):
    '''Generator for exploring a search space'''
    
    Agenda_item=coll.namedtuple('AgendaItem', ['state', 'path'])
    Search_node=coll.namedtuple('SearchNode', ['state', 'path', 'result'])
    
    visited_set=set()
    # maps start_position onto a tuple. In case I forget and send a list...
    agenda=[Agenda_item(start_state, pyr.pvector())]

    while agenda:
        next_agenda_item=agenda.pop(0)
        
        if next_agenda_item.state not in visited_set:

            if fail_fn(next_agenda_item.state):
                yield Search_node(next_agenda_item.state,
                                  next_agenda_item.path,
                                  'Fail')

            elif success_fn(next_agenda_item.state):
                visited_set.add(next_agenda_item.state)
                agenda.extend([Agenda_item(next_state,
                                           next_agenda_item.path.append(next_agenda_item.state))
                               for next_state in daughters_fn(next_agenda_item.state)])
                yield Search_node(next_agenda_item.state,
                                  next_agenda_item.path,
                                  'Success')

            else:
                visited_set.add(next_agenda_item.state)
                agenda.extend([Agenda_item(next_state,
                                           next_agenda_item.path.append(next_agenda_item.state))
                               for next_state in daughters_fn(next_agenda_item.state)])
                yield Search_node(next_agenda_item.state,
                                  next_agenda_item.path,
                                  'None')

In [106]:
def move1_fn(pos):
    return [(pos[0], pos[1]-1), # move up
            (pos[0]-1, pos[1]), # move left
            (pos[0]+1, pos[1]), # move right
            (pos[0], pos[1]+1)] # move down
move1((3, 4))

[(3, 3), (2, 4), (4, 4), (3, 5)]

In [107]:
w=walk_search_space((3, 3),
                    move1_fn,
                    lambda x:False,
                    lambda x:False)

In [108]:
for i in it.islice(w, 0, 20):
    print(i)

SearchNode(state=(3, 3), path=pvector([]), result='None')
SearchNode(state=(3, 2), path=pvector([(3, 3)]), result='None')
SearchNode(state=(2, 3), path=pvector([(3, 3)]), result='None')
SearchNode(state=(4, 3), path=pvector([(3, 3)]), result='None')
SearchNode(state=(3, 4), path=pvector([(3, 3)]), result='None')
SearchNode(state=(3, 1), path=pvector([(3, 3), (3, 2)]), result='None')
SearchNode(state=(2, 2), path=pvector([(3, 3), (3, 2)]), result='None')
SearchNode(state=(4, 2), path=pvector([(3, 3), (3, 2)]), result='None')
SearchNode(state=(1, 3), path=pvector([(3, 3), (2, 3)]), result='None')
SearchNode(state=(2, 4), path=pvector([(3, 3), (2, 3)]), result='None')
SearchNode(state=(5, 3), path=pvector([(3, 3), (4, 3)]), result='None')
SearchNode(state=(4, 4), path=pvector([(3, 3), (4, 3)]), result='None')
SearchNode(state=(3, 5), path=pvector([(3, 3), (3, 4)]), result='None')
SearchNode(state=(3, 0), path=pvector([(3, 3), (3, 2), (3, 1)]), result='None')
SearchNode(state=(2, 1), path=

In [109]:
t='''#######
#E..G.#
#...#.#
#.G.#G#
#######'''

parse_input(t)

{'elves': [(1, 1)],
 'goblins': [(4, 1), (2, 3), (5, 3)],
 'map': [['#', '#', '#', '#', '#', '#', '#'],
  ['#', '.', '.', '.', '.', '.', '#'],
  ['#', '.', '.', '.', '#', '.', '#'],
  ['#', '.', '.', '.', '#', '.', '#'],
  ['#', '#', '#', '#', '#', '#', '#']],
 'walls': [(0, 0),
  (1, 0),
  (2, 0),
  (3, 0),
  (4, 0),
  (5, 0),
  (6, 0),
  (0, 1),
  (6, 1),
  (0, 2),
  (4, 2),
  (6, 2),
  (0, 3),
  (4, 3),
  (6, 3),
  (0, 4),
  (1, 4),
  (2, 4),
  (3, 4),
  (4, 4),
  (5, 4),
  (6, 4)]}

In [116]:
z=parse_input(t)

elf_walk=walk_search_space((1, 1),
                           move1_fn,
                           success_fn=lambda x:x in z['goblins'],
                           fail_fn=lambda x:x in z['walls'])

In [115]:
for i in elf_walk:
    if not i.result=='Fail':
        print(i)

SearchNode(state=(1, 1), path=pvector([]), result='None')
SearchNode(state=(2, 1), path=pvector([(1, 1)]), result='None')
SearchNode(state=(1, 2), path=pvector([(1, 1)]), result='None')
SearchNode(state=(3, 1), path=pvector([(1, 1), (2, 1)]), result='None')
SearchNode(state=(2, 2), path=pvector([(1, 1), (2, 1)]), result='None')
SearchNode(state=(1, 3), path=pvector([(1, 1), (1, 2)]), result='None')
SearchNode(state=(4, 1), path=pvector([(1, 1), (2, 1), (3, 1)]), result='Success')
SearchNode(state=(3, 2), path=pvector([(1, 1), (2, 1), (3, 1)]), result='None')
SearchNode(state=(2, 3), path=pvector([(1, 1), (2, 1), (2, 2)]), result='Success')
SearchNode(state=(5, 1), path=pvector([(1, 1), (2, 1), (3, 1), (4, 1)]), result='None')
SearchNode(state=(3, 3), path=pvector([(1, 1), (2, 1), (3, 1), (3, 2)]), result='None')
SearchNode(state=(5, 2), path=pvector([(1, 1), (2, 1), (3, 1), (4, 1), (5, 1)]), result='None')
SearchNode(state=(5, 3), path=pvector([(1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (

In [118]:
sorted([(x, y) for x in range(3) for y in range(3)])

[(0, 0), (0, 1), (0, 2), (1, 0), (1, 1), (1, 2), (2, 0), (2, 1), (2, 2)]

Next, we need to find a unit's path. Let's have functions `move_elf` and `move_goblin`:

In [46]:
def move_elf(elf_loc, state):

    elf_path=it.filterfalse(lambda x:state['map'][x[0][0]][x[0][1]]=='#',
                            move(elf_loc))
    
    for (pos, path) in it.islice(elf_path, 0, 30):
        print((pos, path))
        
    

In [51]:
for i in it.islice(move((1,1)), 0, 20):
    print(i)

((1, 1), pvector([]))
((1, 0), pvector([(1, 1)]))
((1, 2), pvector([(1, 1)]))
((0, 1), pvector([(1, 1)]))
((2, 1), pvector([(1, 1)]))
((1, -1), pvector([(1, 1), (1, 0)]))
((0, 0), pvector([(1, 1), (1, 0)]))
((2, 0), pvector([(1, 1), (1, 0)]))
((1, 3), pvector([(1, 1), (1, 2)]))
((0, 2), pvector([(1, 1), (1, 2)]))
((2, 2), pvector([(1, 1), (1, 2)]))
((-1, 1), pvector([(1, 1), (0, 1)]))
((3, 1), pvector([(1, 1), (2, 1)]))
((1, -2), pvector([(1, 1), (1, 0), (1, -1)]))
((0, -1), pvector([(1, 1), (1, 0), (1, -1)]))
((2, -1), pvector([(1, 1), (1, 0), (1, -1)]))
((-1, 0), pvector([(1, 1), (1, 0), (0, 0)]))
((3, 0), pvector([(1, 1), (1, 0), (2, 0)]))
((1, 4), pvector([(1, 1), (1, 2), (1, 3)]))
((0, 3), pvector([(1, 1), (1, 2), (1, 3)]))


## Part 2

In [5]:
u=pyr.pvector()
u.append('a')

pvector(['a'])

In [6]:
u

pvector([])